In [ ]:
import ccapi

config         = ccapi.Configuration()
config.url     = "http://localhost:5000"
config.verbose = True

### Building a Constraint-Based Model

This simple example demonstrates how to create a constraint-based model, its metabolites, reactions, genes, etc. into [Cell Collective](https://cellcollective.org).

We’ll use the `3OAS140` reaction from the `STM_1.0 model`:

```
1.0 malACP[c] + 1.0 h[c] + 1.0 ddcaACP[c] → 1.0 co2[c] + 1.0 ACP[c] + 1.0 3omrsACP[c]
```

Begin by importing the ccapi module into your workspace.

In [ ]:
import ccapi

Now, let’s try creating a client object in order to interact with services provided by [Cell Collective](https://cellcollective.org).

In [ ]:
client = ccapi.Client()

Authenticate your client using a ***password flow type authentication*** scheme.

**NOTE**: *Before you can authenticate using ccapi, you must first register an application of the appropriate type on [Cell Collective](https://cellcollective.org). If you do not require a user context, it is read only.*

In [ ]:
client.auth(email = "test@cellcollective.org", password = "test")

#### Creating a Base Model

Create a Base Model using ccapi and instantize it with an authenticated client.

In [ ]:
model = ccapi.Model("Sample Constraint-Based Model", default_type = "metabolic", client = client)
model.save3()

A `ccapi.Model` consists of various `ccapi.ModelVersion` objects that help you build various versions to a model. By default, a `ccapi.Model` provides you a default model version of a boolean type. You can override the same by passing the `default_type`.

In [ ]:
# get the default model version
metabolic      = model.versions[0]
metabolic.name = "Version 1"
metabolic

#### Adding Metabolites to a Constraint-Based Model

First, we need to create a list of metabolite objects for this model.

In [ ]:
# create metabolites
malACP_c      = ccapi.Metabolite("Malonyl-acyl-carrier-protein",
                                 formula     = "C14H22N2O10PRS",
                                 compartment = "c")
ACP_c         = ccapi.Metabolite("acyl-carrier-protein",
                                 formula     = "C11H21N2O7PRS",
                                 compartment = "c")
h_c           = ccapi.Metabolite("H",
                                 formula     = "H",
                                 compartment = "c")
ddcaACP_c     = ccapi.Metabolite("Dodecanoyl-ACP-n-C120ACP",
                                 formula     = "C23H43N2O8PRS",
                                 compartment = "c")
co2_c         = ccapi.Metabolite("CO2",
                                 formula     = "CO2",
                                 compartment = "c")
omrsACP_c     = ccapi.Metabolite("3-Oxotetradecanoyl-acyl-carrier-protein",
                                 formula     = "C25H45N2O9PRS",
                                 compartment = "c")

Now let us add a list of components to our Constraint-Based Model.

In [ ]:
metabolic.add_metabolites(malACP_c, ACP_c, h_c, ddcaACP_c, co2_c, omrsACP_c)
metabolic

In [ ]:
metabolic.metabolites

#### Saving a Model

Ensure you save your model in order to commit your work.

In [ ]:
model.save3()

In [ ]:
metabolic.metabolites

#### Adding Reactions to a Model

First, we create a `ccapi.Reaction` object.

In [ ]:
reaction = ccapi.Reaction("3 oxoacyl acyl carrier protein synthase n C140",
                         subsystem   = "Cell Envelope Biosynthesis",
                         lower_bound = 0,
                         upper_bound = 1000)

Adding metabolites to a reaction requires using a dictionary of the metabolites and their stoichiometric coefficients. A group of metabolites can be added all at once, or they can be added one at a time.

In [ ]:
reaction.add_metabolites({
    malACP_c:  -1,
    h_c:       -1,
    ddcaACP_c: -1,
    co2_c:      1,
    ACP_c:      1,
    omrsACP_c:  1
})

We will now add a reaction to the model

In [ ]:
metabolic.add_reaction(reaction)

In [ ]:
model.save3()

In [ ]:
metabolic.reactions

#### Adding a Gene Reaction Rule to a Reaction

In order to add a Gene Reaction Rule to a Reaction, create genes as follows.

In [ ]:
STM2378 = ccapi.Gene("STM2378")
STM1197 = ccapi.Gene("STM1197")

Let's consider a Gene Reaction Rule as follows

```
STM2378 or STM1197
```

You can add the same using the Regulatory Mechanism provided for Boolean Models as follows:

In [ ]:
reaction.add_regulators(
    ccapi.PositiveRegulator(STM2378),
    ccapi.PositiveRegulator(STM1197)
)